# History of Euro vs Dollar [ANIMATED!]
In this notebook we are going to study the history of the Euro vs US Dollar using our generated plot. There are 7 interesting highlights in the history of the Euro let's take a look at them:
* 1- The currency was introduced in non-physical form (traveller's cheques, electronic transfers, banking, etc.) at midnight on 1 January 1999 [[source]](https://en.wikipedia.org/wiki/History_of_the_euro#Launch)
* 2- Transition from national legacy currencies to euro 2002 [[source]](https://en.wikipedia.org/wiki/History_of_the_euro#Aftermath)
* 3- Global financial crisis 2008 [[source]](https://en.wikipedia.org/wiki/History_of_the_euro#Recession_era)
* 4- Greek Crisis 2010 [[source]](https://www.thebalance.com/what-is-the-greece-debt-crisis-3305525#:~:text=Why%20Greece%20Didn't%20Leave%20the%20Eurozone&text=Without%20the%20austerity%20measures%2C%20the,lowered%20its%20euro%20exchange%20rate.)
* 5- Euro plunging for many reasons Much of has been about the strength of the Dollar 2014. [[source]](https://money.cnn.com/2015/01/05/investing/euro-slump-deepens/index.html)
* 6- Quantitative Easing start in Europe 2015 [[source]](https://www.reuters.com/article/us-eurozone-ecb-qe-idUSKBN1OB1SM)
* 7- Coronavirus Pandemic 2020 [[source]](https://www.statista.com/statistics/1102209/coronavirus-cases-development-europe/)

In [ ]:
# Install celluloid for animation 
# Documentation: https://pypi.org/project/celluloid/
!pip install celluloid 

In [ ]:
# Load packages
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime
from celluloid import Camera
from IPython.display import Image
%matplotlib inline

In [ ]:
# Read data
PATH = '../input/euro-exchange-daily-rates-19992020/euro-daily-hist_1999_2020.csv'
df = pd.read_csv(PATH)

In [ ]:
# Show first 5 rows for data exploration
df.head()

In [ ]:
# Understanding of our columns and their dtypes
df.info()

# Clean
Below, we do the following:

* We rename the `[US dollar ]` and `Period\Unit:` columns to something easier to type — us_dollar and time.
* We change the time column to a datetime data type.
* We sort the values by time in ascending order.
* We reset the index (and drop the initial index).
* Isolate `time` & `us_dollar` in a separate df since this is our area of focus.
* Drop rows with `us_dollar` value = `-`
* Convert the `us_dollar` column to a float data type.

## Columns renaming

In [ ]:
df.rename(columns={'[US dollar ]':'us_dollar', 'Period\\Unit:':'time'}, inplace=True)

# Test
df[['time','us_dollar']].sample()

## Change the time column to a datetime data type

In [ ]:
df['time'] = pd.to_datetime(df.time)

# Test
df.time.dtype

## Sort the values by time in ascending order & reset index.

In [ ]:
df = df.sort_values('time').reset_index(drop=True)

# Test
df.head()

## Isolate `time` & `us_dollar` in a separate df

In [ ]:
usd_df = df[['time','us_dollar']].copy()

# Test
usd_df.sample()

In [ ]:
usd_df.us_dollar.value_counts()

## Drop rows with `us_dollar` value = `-`

In [ ]:
usd_df.drop(index=usd_df[usd_df['us_dollar']=='-'].index, inplace=True)

# Test
usd_df.us_dollar.value_counts()

## Convert the `us_dollar` column to a float data type.

In [ ]:
usd_df.us_dollar = usd_df.us_dollar.astype('float')

# Test
usd_df.info()

---
# Analysis
Now that we're finished cleaning the data, we'll generate a line plot to visualize the evolution of the euro-dollar exchange rate.

In [ ]:
fig, ax = plt.subplots()
ax.plot(usd_df.time, usd_df.us_dollar)

The resulting figure has many wiggles representing the daily variation of the exchange rate. However, we are only interested in long term variation so next we are going to apply rolling mean with rolling window of 30 days showing variation of exchange rate every 30 days.

In [ ]:
usd_df['rolling_mean_30'] = usd_df.us_dollar.rolling(30).mean()

# Test
usd_df.tail()

In [ ]:
# Replot vs rolling mean
fig, ax = plt.subplots()
ax.plot(usd_df.time, usd_df.rolling_mean_30)

# Producing our story

In [ ]:
# Generating figure
fig , axs = plt.subplots(7,1,figsize=(8,24))

# Initializing our figures parameters
starts=[0, 782, 2455, 2807, 3935, 4175, 5440]
ends =[782, 2455, 2600, 2940, 4175, 5440, -1]
wys = [1.53, 1.48, 1.53, 1.53, 1.48, 1.48, 1.48]
texts = ['Euro Introduction', 'Transition From Legacy\n   Currencies to Euro', 'Global Financial Crisis',
       'Greek Crisis', '   Euro Plunging\nVS Mighty Dollar', 'Quantitative Easing\n          Policy',
       'Coronavirus\n  Pandemic']
sizes=[10, 8, 10, 10, 8, 8, 8]

# looping over figures
for start, end, wy, text, size, ax in zip(starts, ends, wys, texts, sizes, axs):
    # Background figure
    ax.plot(usd_df.time, usd_df.rolling_mean_30, color='#b00b1e', alpha=0.25)
    ax.axhline(y=1, color='#898989', alpha=0.2, xmax=0.97)
    ax.spines['right'].set_visible(False)
    ax.spines['top'].set_visible(False)
    ax.tick_params(left=False)
    
    # Foreground figure
    ax.plot(usd_df.time[start:end], usd_df.rolling_mean_30[start:end], color='#b00b1e')
    ax.text(x=datetime.strptime('1999', '%Y'), y=wy, s=text, size=size,
             bbox=dict(facecolor='none', edgecolor='black', boxstyle='round,pad=0.2'))
    
# Title & subtitle
axs[0].text(x=datetime.strptime('1998', '%Y'), y=1.8, s='Euro more valuable than Dollar:', weight='bold', size=18)
axs[0].text(x=datetime.strptime('1998', '%Y'), y=1.71, s='History of the Euro currency', size=15)

# Signature
axs[6].text(x=datetime.strptime('1997', '%Y'), y=0.59, s='©Youssef Etman'+' '*90+'Source:thebalance.com',
            backgroundcolor='#000000', color='#ffffff');

The figure above is too long and might be annoying for some readers, Next we are going to generate an animated GIF that summarizes all of the above figures.

In [ ]:
# Initialize plot and animation camera
fig1, (ax1, ax2) = plt.subplots(1,2, figsize=(14,4))
camera = Camera(fig1)

# Looping over figures as above
for start, end, wy, text, size in zip(starts, ends, wys, texts, sizes):
    for i in range(0,5):
        # Main still figure
        ax1.plot(usd_df.time, usd_df.rolling_mean_30, color='#b00b1e')
        ax1.axhline(y=1, color='#898989', alpha=0.4, xmax=0.97)
        ax1.text(x=datetime.strptime('2007', '%Y'), y=1.02, s='Euro stronger ↑', color='#898989')
        ax1.text(x=datetime.strptime('2007', '%Y'), y=0.96, s='Dollar stronger ↓', color='#898989')
        ax1.spines['right'].set_visible(False)
        ax1.spines['top'].set_visible(False)
        ax1.tick_params(left=False)
        ax1.set_ylabel('Eur / USD')
    
        ax2.plot(usd_df.time, usd_df.rolling_mean_30, color='#b00b1e', alpha=0.25)
        ax2.axhline(y=1, color='#898989', alpha=0.2, xmax=0.97)
        ax2.spines['right'].set_visible(False)
        ax2.spines['top'].set_visible(False)
        ax2.tick_params(left=False)
        
        # Title
        ax1.text(x=datetime.strptime('1998', '%Y'), y=1.65, s='History of the Euro currency', size=15)
        
        # Signature
        ax1.text(x=datetime.strptime('1997', '%Y'), y=0.7, s='©Youssef Etman'+' '*195+'Source:thebalance.com',
        backgroundcolor='#000000', color='#ffffff')

        # Dynamic figure
        ax2.plot(usd_df.time[start:end], usd_df.rolling_mean_30[start:end], color='#b00b1e')
        ax2.text(x=datetime.strptime('1999', '%Y'), y=wy, s=text, size=size, bbox=dict(facecolor='none', 
                                                                                        edgecolor='black', 
                                                                                        boxstyle='round,pad=0.2'))
        camera.snap()
    
# Compile & save animation
animation = camera.animate()
animation.save('euro_hist.gif')

# Clear figure
plt.clf()

# Display saved GIF
Image(url='euro_hist.gif')